In [ ]:
import os
os.mjdir('deploy')
os.mkdir('deploy/model')

In [ ]:
!gsutil cp gs://curso-platzi-cv/fine_turned_model.zip deploy/model/fine_turned_model.zip
!gsutil cp gs://curso-platzi-cv/label_map.pbtxt deploy/model/label_map.pbtxt

In [ ]:
!gsutil cp gs://curso-platzi-cv/dist/centroidtracker.py deploy/centroidtracker.py
!gsutil cp gs://curso-platzi-cv/dist/trackableobject.py deploy/trackableobject.py

In [ ]:
import zipfile
local_zip = 'deploy/model/fine_turned_model.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('deploy/model/fine_turned_model')
zip_ref.close()

In [ ]:
%%writefile deploy/app.py

from flask import Flask, requests
from prediction import smartcities
import base64

app = Flask(__name__)

@app.route('/', methods = ['POST'])
def parse_request():
    request_data = requests.get_json()
    videoBase64 = request_data['video']
    sc = smartcities()
    response_64 = sc.predict(videoBase64)
    return jsonify(response_64)

if __name__ == '__main__':
    app.run(debug=True, host = '0.0.0.0', port = 8000)

In [ ]:
%%writefile deploy/prediction.py

import tensorflow as tf
import numpy as np

import imutils
import time
import dlib
import cv2
import base64
from PIL import Image
import matplotlib.pyplot as plt
from imutils.video import VideoStream
from imutils.video import FPS
from centroidtracker import CentroidTracker
from trackableobject import TrackableObject

class smartcities:
  def __init__(self):
    detect_fn = tf.saved_model.load("model/fine_tuned_model/saved_model")
    self.detect_fn = detect_fn

  def predict(self, image_64_decode):

    PATH_VIDEO = "/tmp/video_in.mp4"
    video_result = open(PATH_VIDEO, "wb")
    video_result.write(base64.b64decode(image_64_decode))
    #video_result.close()

    PATH_OUTPUT = "/tmp/output.mp4"

    SKIP_FPS = 30

    TRESHOLD = 0.5

    vs = cv2.VideoCapture(PATH_VIDEO)

    writer = None

    W = int(vs.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(vs.get(cv2.CAP_PROP_FRAME_HEIGHT))

    ct = CentroidTracker(maxDisappeared= 40, maxDistance = 50)

    trackers = []
    trackableObjects = {}

    totalFrame = 0
    totalDown = 0
    totalUp = 0

    DIRECTION_PEOPLE = True

    POINT = [0, int((H/2)-H*0.1), W, int(H*0.1)]

    fps = FPS().start()

    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    writer = cv2.VideoWriter(PATH_OUTPUT, fourcc, 20.0, (W, H), True)

    while True:
      ret, frame = vs.read()

      if frame is None:
        break

      status = "Waiting"
      rects = []

      if totalFrame % SKIP_FPS == 0:
        status = "Detecting"
        trackers = []

        image_np = np.array(frame)

        input_tensor = tf.convert_to_tensor(image_np)
        input_tensor = input_tensor[tf.newaxis, ...]

        detections = self.detect_fn(input_tensor)

        detection_scores = np.array(detections["detection_scores"][0])

        detection_clean = [x for x in detection_scores if x >= TRESHOLD]

        for x in range(len(detection_clean)):
          idx = int(detections['detection_classes'][0][x])

          ymin, xmin, ymax, xmax = np.array(detections['detection_boxes'][0][x])
          box = [xmin, ymin, xmax, ymax] * np.array([W,H, W, H])

          (startX, startY, endX, endY) = box.astype("int")

          tracker = dlib.correlation_tracker()
          rect = dlib.rectangle(startX, startY, endX, endY)
          tracker.start_track(frame, rect)

          trackers.append(tracker)
      else:

        for tracker in trackers:
          status = "Tracking"

          tracker.update(frame)
          pos = tracker.get_position()

          startX = int(pos.left())
          startY = int(pos.top())
          endX = int(pos.right())
          endY = int(pos.bottom())

          rects.append((startX, startY, endX, endY))


      cv2.rectangle(frame, (POINT[0], POINT[1]), (POINT[0]+ POINT[2], POINT[1] + POINT[3]), (255, 0, 255), 2)

      objects = ct.update(rects)

      for (objectID, centroid) in objects.items():

        to = trackableObjects.get(objectID, None)
        if to is None:
          to = TrackableObject(objectID, centroid)

        else:
          y = [c[1] for c in to.centroids]
          direction = centroid[1] - np.mean(y)
          to.centroids.append(centroid)
          if not to.counted:
            if centroid[0] > POINT[0] and centroid[0] < (POINT[0]+ POINT[2]) and centroid[1] > POINT[1] and centroid[1] < (POINT[1]+POINT[3]):
              if DIRECTION_PEOPLE:
                if direction >0:
                  totalUp += 1
                  to.counted = True
                else:
                  totalDown +=1
                  to.counted = True
              else:
                if direction <0:
                  totalUp += 1
                  to.counted = True
                else:
                  totalDown +=1
                  to.counted = True

        trackableObjects[objectID] = to

        text = "ID {}".format(objectID)
        cv2.putText(frame, text, (centroid[0]-10, centroid[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
        cv2.circle(frame, (centroid[0], centroid[1]), 4, (0,255,0), -1)

      info = [
              ("Subiendo", totalUp),
              ("Bajando", totalDown),
              ("Estado", status),
        ]

        for (i, (k,v)) in enumerate(info):
          text = "{}: {}".format(k,v)
          cv2.putText(frame, text, (10, H - ((i*20) + 20)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

        writer.write(frame)
        totalFrame += 1
        fps.update()


    fps.stop()

    print("Tiempo completo {}".format(fps.elapsed()))
    print("Tiempo aproximado por frame {}".format(fps.fps()))


    writer.release()
    vs.release()

    video = open(PATH_OUTPUT, "rb")
    video_read = video.read()
    image_64_encode = base64.b64encode(video_read)
    image_64_encode_return = {1:image_64_encode}
    return image_64_encode_return


In [ ]:
%%writefile deploy/requirements.txt
tf_slim
tf-models-official
lvis
Cython
contextlib2
pillow
lxml
matplotlib
pycocotools
flask
imutils
numpy
dlib
opencv-contrib-python

In [ ]:
%%writefile deploy/Dockerfile

FROM python:3.7
COPY . /app
RUN apt-get update && yes | apt-get upgrade
WORKDIR /app
RUN mkdir -p /tensorflow/models
RUN apt-get install -y git python3-pip
RUN pip install --upgrade pip
RUN pip install tensorflow
RUN apt-get install -y protobuf-compiler python3-pil python3-lxml
RUN pip install matplotlib
RUN git clone https://github.com/tensorflow/models.git /tensorflow/models
RUN apt-get update && apt-get install -y cmake
RUN apt-get install ffmpeg libsm6 libxext6  -y

WORKDIR /tensorflow/models/research
RUN protoc object_detection/protos/*.proto --python_out=.
RUN export PYTHONPATH=$PYTHONPATH:pwd:pwd/slim

WORKDIR /app
RUN pip install -r requirements.txt
CMD ["python3", "./app.py"]